## 6주 zscore, cdf 계산 과제 - 201810803 장서령 
#### 문제: zscore, cdf 계산

성적데이터는 n이 적지만, 정규분포를 이룬다고 가정하자.

* 1-1 성적데이터로 DataFrame을 생성.
* 1-2 zscore 컬럼을 생성.
(zscore를 계산하려면, 평균과 표준편차를 알아야 한다.
계산식에 F함수를 직접 사용하면 오류가 발생한다. 따로 평균과 표준편차를 구해서 계산식에서 사용해야 한다.)
* 1-3 cdf 컬럼을 생성.
scipy.stats.norm.cdf() 함수는 데이터타입을 float로 맞추어 주어야 한다.
cdf는 평균=0, 표준편차=1을 기본 값으로 누적확률을 계산한다.

In [2]:
import os
import pyspark

os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"
myConf=pyspark.SparkConf()
spark=pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

In [3]:
marks=[
    "김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"
]

### 1-1 성적데이터로 DataFrame 생성

In [5]:
# 성적데이터를 rdd로 생성
marksRdd=spark.sparkContext.parallelize(marks)

In [6]:
#rdd에서 ,으로 str 분리 후 rdd로 dataframe 생성하기
_marksRdd = marksRdd.map(lambda l: l.split(","))
marksDF=spark.createDataFrame(_marksRdd)

In [7]:
marksDF=marksDF.withColumnRenamed('_1','name').withColumnRenamed('_2','subject').withColumnRenamed('_3','mark')
marksDF.printSchema()

root
-- name: string (nullable = true)
-- subject: string (nullable = true)
-- mark: string (nullable = true)

In [8]:
marksDF=marksDF.withColumn("mark", marksDF.mark.cast("integer"))
marksDF.printSchema()
marksDF.show()

root
-- name: string (nullable = true)
-- subject: string (nullable = true)
-- mark: integer (nullable = true)

+------+--------+----+
 name| subject|mark|
+------+--------+----+
김하나| English| 100|
김하나| Math| 80|
임하나| English| 70|
임하나| Math| 100|
김갑돌| English| 82|
김갑돌| Math| 98|
+------+--------+----+

### 1-2 zscore 컬럼을 생성
(zscore를 계산하려면, 평균과 표준편차를 알아야 한다.
계산식에 F함수를 직접 사용하면 오류가 발생한다. 따로 평균과 표준편차를 구해서 계산식에서 사용해야 한다.)

In [10]:
# rdd에서 mark 값만 추출하기
marksdata=marksRdd.map(lambda x:x.split(',')).map(lambda x: (float(x[2]))).collect()
print (marksdata)

[100.0, 80.0, 70.0, 100.0, 82.3, 98.5]

In [11]:
# 평균 구하기
import numpy as np
mean=np.mean(marksdata)
print ("numpy mean of mark = {:.3f}".format(mean))

numpy mean of mark = 88.467

In [12]:
# 분산구하기
deviation =(np.sqrt(sum( [ (marksdata-mean)**2 for x in marksdata ] )/len(marksdata))) # ddof=0
print(deviation)

[11.53333333 8.46666667 18.46666667 11.53333333 6.16666667 10.03333333]

In [13]:
# zscore 구하기 - 값만 출력됨
#sx=np.std(marksdata,ddof=1)
#for x in marksdata:
#   zscore=(x-mean)/sx
#   print (zscore, end= ' ')

In [14]:
# zscore 구해서 리스트에 저장하기
sx=np.std(marksdata,ddof=1)
zscorelist = []
for x in marksdata:
   zscore=(x-mean)/sx
   zscorelist.append(zscore)
print (zscorelist)

[0.9020148231647499, -0.662172731456202, -1.444266508766678, 0.9020148231647499, -0.4822911626747928, 0.7847007565681785]

In [15]:
# zscorelist를 dataframe에 추가하기 
import pandas as pd
marksDF_P = marksDF.toPandas()
# DataFrame을 pandas로 변환 후, zscore 추가하기 
marksDF_P.loc[:,'zscore'] = zscorelist
print(marksDF_P)

name subject mark zscore
0 김하나 English 100 0.902015
1 김하나 Math 80 -0.662173
2 임하나 English 70 -1.444267
3 임하나 Math 100 0.902015
4 김갑돌 English 82 -0.482291
5 김갑돌 Math 98 0.784701

In [16]:
# pandsDF을 다시 DataFrame으로 바꾸기
marksDF_z = spark.createDataFrame(marksDF_P)
marksDF_z.show()

+------+--------+----+-------------------+
 name| subject|mark| zscore|
+------+--------+----+-------------------+
김하나| English| 100| 0.9020148231647499|
김하나| Math| 80| -0.662172731456202|
임하나| English| 70| -1.444266508766678|
임하나| Math| 100| 0.9020148231647499|
김갑돌| English| 82|-0.4822911626747928|
김갑돌| Math| 98| 0.7847007565681785|
+------+--------+----+-------------------+

### 1-3 cdf 컬럼을 생성
scipy.stats.norm.cdf() 함수는 데이터타입을 float로 맞추어 주어야 한다.
cdf는 평균=0, 표준편차=1을 기본 값으로 누적확률을 계산한다.

In [18]:
# 데이터타입을 float로 맞추기 위해 mark의 데이터 타입을 float로 바꾸어주기
marksDF_zF = marksDF_z.withColumn("mark", marksDF_z['mark'].cast("float"))
marksDF_zF.printSchema()
marksDF_zF.show()

root
-- name: string (nullable = true)
-- subject: string (nullable = true)
-- mark: float (nullable = true)
-- zscore: double (nullable = true)

+------+--------+-----+-------------------+
 name| subject| mark| zscore|
+------+--------+-----+-------------------+
김하나| English|100.0| 0.9020148231647499|
김하나| Math| 80.0| -0.662172731456202|
임하나| English| 70.0| -1.444266508766678|
임하나| Math|100.0| 0.9020148231647499|
김갑돌| English| 82.0|-0.4822911626747928|
김갑돌| Math| 98.0| 0.7847007565681785|
+------+--------+-----+-------------------+

In [19]:
# cdf 구하기
from scipy.stats import norm

# zscorelist를 인자로 받아서 cdf 값을 구해야 함
cdflist = []
for x in zscorelist:
  cdfdata=norm.cdf(x, loc=0, scale=1)
  cdflist.append(cdfdata)
print(cdflist)

[0.8164755032306112, 0.2539302639589459, 0.07433201139097219, 0.8164755032306112, 0.31479956212721416, 0.7836854804484269]

In [20]:
# zscorelist가 추가된 pandasdf에 cdflist 추가하기 
import pandas as pd
marksDF_P = marksDF.toPandas()
# DataFrame을 pandas로 변환 후, zscore 추가하기 
marksDF_P.loc[:,'zscore'] = zscorelist
marksDF_P.loc[:,'cdf'] = cdflist
print(marksDF_P)

name subject mark zscore cdf
0 김하나 English 100 0.902015 0.816476
1 김하나 Math 80 -0.662173 0.253930
2 임하나 English 70 -1.444267 0.074332
3 임하나 Math 100 0.902015 0.816476
4 김갑돌 English 82 -0.482291 0.314800
5 김갑돌 Math 98 0.784701 0.783685

In [21]:
# pandsDF을 다시 DataFrame으로 바꾸기
marksDF_zFc = spark.createDataFrame(marksDF_P)
marksDF_zFc.show()

+------+--------+----+-------------------+-------------------+
 name| subject|mark| zscore| cdf|
+------+--------+----+-------------------+-------------------+
김하나| English| 100| 0.9020148231647499| 0.8164755032306112|
김하나| Math| 80| -0.662172731456202| 0.2539302639589459|
임하나| English| 70| -1.444266508766678|0.07433201139097219|
임하나| Math| 100| 0.9020148231647499| 0.8164755032306112|
김갑돌| English| 82|-0.4822911626747928|0.31479956212721416|
김갑돌| Math| 98| 0.7847007565681785| 0.7836854804484269|
+------+--------+----+-------------------+-------------------+